In [18]:
%matplotlib inline
# %load_ext autoreload
# %autoreload 2

import sys; sys.path.insert(0, '../')
import pickle as pkl
import numpy as np
import pandas as pd
from copy import deepcopy
import mne
import seaborn as sns
import matplotlib.pyplot as plt
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet import forward

plot_params = dict(surface='white', hemi='both', verbose=0)

# Forward Model

In [2]:
info = forward.get_info()
info['sfreq'] = 100
fwd = forward.create_forward_model(info=info)
fwd_free = forward.create_forward_model(info=info, fixed_ori=False)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    3.3s remaining:    3.3s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    3.6s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    3.6s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(

# Load Models

In [3]:
lstm_standard = util.load_net('models/LSTM Medium_1-1000points_standard-cosine-mse_0')
# lstm_noise = util.load_net('models/LSTM Medium_1-1000points_noise-cosine-mse_0')

dense_standard = util.load_net('models/Dense Medium_1-1000points_standard-cosine-mse_0')
# dense_noise = util.load_net('models/Dense Medium_1-1000points_noise-cosine-mse_0')

convdip_standard = util.load_net('models/ConvDip Medium_1-1000points_standard-cosine-mse_0')
# convdip_noise = util.load_net('models/ConvDip Medium_1-1000points_noise-cosine-mse_0')

# models = [lstm_standard, lstm_noise, dense_standard, dense_noise, convdip_standard, convdip_noise]
# model_names = ['LSTM Standard', 'LSTM Noise', 'Dense Standard', 'Dense Noise', 'ConvDip Standard', 'ConvDip Noise']
models = [lstm_standard, dense_standard, convdip_standard]
model_names = ['LSTM Standard', 'Dense Standard', 'ConvDip Standard']

# models = [lstm_noise, dense_noise, convdip_noise]
# model_names = ['LSTM Noise', 'Dense Noise', 'ConvDip Noise']


# Test ability to process long sequences

In [4]:
%load_ext autoreload
%autoreload 2

settings_eval = dict(duration_of_trial=100, method='standard', number_of_sources=10)
sim_test = Simulation(fwd, info, settings=settings_eval).simulate(2)
idx = 0
durs = [100, 50, 10, 1, 0.1, 0.02]
# durs = [np.random.uniform(0.01, 100) for _ in range(5)]
for net, name in zip(models, model_names):
    for dur in durs:
        sim_test.crop(tmax=dur)
        prediction = net.predict(sim_test)
        error = util.batch_nmse(sim_test.source_data[idx].data, prediction[idx].data)
        r = util.batch_corr(sim_test.source_data[idx].data, prediction[idx].data)
        title = f'{dur}: error: {error:.4}, r: {r:.2f}\n'
        print(f"{name}:")
        print(title)


Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 38.93it/s]


source data shape:  (1284, 10000) (1284, 10000)


100%|██████████| 2/2 [00:01<00:00,  1.09it/s]
c:\Users\Lukas\Documents\projects\esinet\evaluate\..\esinet\simulation.py:741: RuntimeWarning: tmax is not in epochs time interval. tmax is set to epochs.tmax
  cropped_eeg = self.eeg_data[i].crop(tmin=tmin, tmax=tmax, include_tmax=include_tmax, verbose=verbose)


LSTM Standard:
100: error: 0.01281, r: 0.49

LSTM Standard:
50: error: 0.01313, r: 0.48



KeyboardInterrupt: 

# Plot single ground truth and predictions

In [20]:
import seaborn as sns
%matplotlib qt
sns.reset_orig()

%load_ext autoreload
%autoreload 2

plot_params = dict(surface='white', hemi='both', verbose=0, 
    clim=dict(kind='percent', pos_lims=[20, 30, 100]))

settings_eval = dict(duration_of_trial=1, method='standard')
# settings_eval = dict(duration_of_trial=1, method='noise', exponent=3)

# Simulate new data
# sim_test = Simulation(fwd, info, settings=settings_eval).simulate(2)
snr = sim_test.simulation_info['target_snr'].values[0]
# print(sim_test.simulation_info)
idx = 0
# Predict sources using the esinet models
predictions = [model.predict(sim_test) for model in models]

# # Predict sources with classical methods
# eLORETA
# prediction_elor_data = util.wrap_mne_inverse(fwd, sim_test, snr=snr)[idx].data.astype(np.float32)
# prediction_elor = deepcopy(predictions[0][0])
# prediction_elor.data = prediction_elor_data / np.abs(np.max(prediction_elor_data))
# # MNE
# prediction_mne_data = util.wrap_mne_inverse(fwd, sim_test, method='MNE', snr=snr)[idx].data.astype(np.float32)
# prediction_mne = deepcopy(predictions[0][0])
# prediction_mne.data = prediction_mne_data / np.abs(np.max(prediction_mne_data))
# # Beamformer
# prediction_lcmv_data = util.wrap_mne_inverse(fwd_free, sim_test, method='beamformer', snr=snr)[idx].data.astype(np.float32)
# prediction_lcmv = deepcopy(predictions[0][0])
# prediction_lcmv.data = prediction_lcmv_data / np.abs(np.max(prediction_lcmv_data))

# Get predictions and names in order
# predictions.append([prediction_elor])
# predictions.append([prediction_mne])
# predictions.append([prediction_lcmv])

# model_names.append('eLORETA')
# model_names.append('MNE')
# model_names.append('Beamformer')

# Plot True Source
brain = sim_test.source_data[idx].plot(**plot_params)
brain.add_text(0.1, 0.9, f'Ground Truth {sim_test.simulation_info.number_of_sources.values[0]} sources, snr={snr:.1f}', 'title')
# Plot True EEG
evoked = sim_test.eeg_data[idx].average()
# evoked.plot()
evoked.plot_topomap(title='Ground Truth')
# evoked = util.get_eeg_from_source(sim_test.source_data[idx], fwd, info, tmin=0.)
# evoked.plot_topomap(title='Ground Truth Noiseless')

model_selection = model_names#['LSTM Noise', 'LSTM Noise', 'eLORETA']
# Plot predicted sources
for model_name, prediction in zip(model_names, predictions):
    
    if not any([model_name.lower() in model_select.lower() for model_select in model_selection]):
        continue
    error = util.batch_nmse(sim_test.source_data[idx].data, prediction[idx].data)
    r = util.batch_corr(sim_test.source_data[idx].data, prediction[idx].data)
    
    brain = prediction[idx].plot(**plot_params)

    title = f'{model_name}, error: {error:.4}, r: {r}'
    print(title)
    brain.add_text(0.1, 0.9, title, 'title')
    # Plot predicted EEG
    evoked_esi = util.get_eeg_from_source(prediction[idx], fwd, info, tmin=0.)
    evoked_esi.plot_topomap(title=model_name)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
interpolating for convdip...


2it [00:00,  8.68it/s]


LSTM Noise, error: 0.06755, r: 0.1009702788203332
Dense Noise, error: 0.03129, r: 0.21136470276728508
ConvDip Noise, error: 0.04425, r: 0.18692590609561874


  File "C:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


Using control points [1.73811822e-11 2.90359280e-11 5.45398807e-11]
Using control points [1.73811822e-11 2.90359280e-11 5.45398807e-11]
Using control points [0.0000000e+00 0.0000000e+00 6.1521689e-10]


  File "C:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'
  File "C:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


Using control points [1.84609501e-11 2.60222306e-11 4.34710364e-11]


  File "C:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


## Create or Load Evaluation Set

In [142]:
# n_samples = 1000
# duration_of_trial = (0.01, 2)
# method = 'standard'
# settings = dict(duration_of_trial=duration_of_trial, method=method)
# sim_test = Simulation(fwd, info, verbose=False, settings=settings).simulate(n_samples=n_samples)
# if type(duration_of_trial) == tuple:
#     sim_test.save(f'simulations\\sim_test_{n_samples}_{int(duration_of_trial[0]*100)}-{int(duration_of_trial[1]*100)}points_{method}.pkl')
# else:
#     sim_test.save(f'simulations\\sim_test_{n_samples}_{int(duration_of_trial*100)}points_{method}.pkl')

# or Load
with open(f'simulations\\sim_test_1000_1-200points_standard.pkl', 'rb') as f:
    sim_test = pkl.load(f)

In [55]:
# %load_ext autoreload
# %autoreload 2

# from esinet.evaluate import eval_mean_localization_error, eval_nmse, eval_auc, eval_mse
# from esinet.util import wrap_mne_inverse
# from scipy.spatial.distance import cdist
# from tqdm.notebook import tqdm
# from joblib import Parallel, delayed

# %load_ext autoreload
# %autoreload 2
# model_names_tmp = deepcopy(model_names)
# # Predict
# print('predict esinets...')
# predictions = [model.predict(sim_test) for model in models]

# print('predict elor')
# pred_elor = wrap_mne_inverse(fwd, sim_test, method='eLORETA')
# model_names_tmp.append('eLORETA')
# predictions.append(pred_elor)

# print('predict MNE')
# pred_mne = wrap_mne_inverse(fwd, sim_test, method='MNE')
# model_names_tmp.append('MNE')
# predictions.append(pred_mne)

# print('predict LCMV')
# pred_lcmv = wrap_mne_inverse(fwd_free, sim_test, method='beamformer', parallel=False)
# model_names_tmp.append('LCMV')
# predictions.append(pred_lcmv)

# pos = util.unpack_fwd(fwd)[2]
# argsorted_distance_matrix = np.argsort(cdist(pos, pos), axis=-1)

# metrics = dict()
# true_sources = np.concatenate([src.data for src in sim_test.source_data], axis=1).T

for prediction, model_name in tqdm(zip(predictions[3:], model_names_tmp[3:])):
    print('\n', model_name, ':\n')
     
    predicted_sources = np.concatenate([src.data for src in prediction], axis=1).T

    print('mle calculation....')
    mean_localization_errors = [eval_mean_localization_error(true_source, predicted_source, pos, argsorted_distance_matrix=argsorted_distance_matrix) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]
    
    print('auc calculation....')
    # aucs_combined = [eval_auc(true_source, predicted_source, pos, epsilon=0.25, n_redraw=5) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]
    aucs_combined = Parallel(n_jobs=-1, backend='loky') \
        (delayed(eval_auc)(true_source, predicted_source, pos, epsilon=0.25, n_redraw=5)
        for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources)))
    print('nmse calculation....')
    nmses = [eval_nmse(true_source, predicted_source) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]
    print('mse calculation....')
    mses = [eval_mse(true_source, predicted_source) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]

    aucs_far = [auc[1] for auc in np.array(aucs_combined)]
    aucs_close = [auc[0] for auc in np.array(aucs_combined)]
    aucs_combined = [np.nanmean([auc[0], auc[1]]) for auc in np.array(aucs_combined)]

    metric = pd.DataFrame(dict(
        mean_localization_errors=mean_localization_errors,
        aucs_combined=aucs_combined,
        aucs_far=aucs_far,
        aucs_close=aucs_close,
        nmses=nmses,
        mses=mses
        )
    )
    metric.name = model_name
    metrics[model_name] = metric
    


with open(f'results\\metrics_{len(true_sources)}_1-200points_standard.pkl', 'wb') as f:
    pkl.dump([metrics, sim_test.simulation_info], f)

0it [00:00, ?it/s]


 eLORETA :

mle calculation....


0it [00:00, ?it/s]

auc calculation....


0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

nmse calculation....


0it [00:00, ?it/s]

mse calculation....


0it [00:00, ?it/s]

## Load Metrics

In [132]:
with open(f'results\\metrics_101947_1-200points_standard.pkl', 'rb') as f:
    [metrics, simulation_info] = pkl.load(f)


# with open(f'simulations\\sim_test_5000_10points.pkl', 'rb') as f:
#     sim_lstm_test = pkl.load(f)

## Prepare Metrics

In [160]:
dfs = [df[1] for df in list(metrics.items())]

for i, (key, df) in enumerate(metrics.items()):
    dfs[i]['method'] = [key]*df.shape[0]
    dfs[i]['sample_id'] = np.arange(df.shape[0])
df_aio = pd.concat(dfs)
df_aio.head()

,mean_localization_errors,aucs_combined,aucs_far,aucs_close,nmses,mses,method,sample_id
0,23.515424,0.967474,0.999308,0.935640,0.008361+0.000000j,2.779794e-19+0.000000e+00j,LSTM Standard,0
1,26.670565,0.796694,0.893222,0.700167,0.009603+0.000000j,8.953524e-19+0.000000e+00j,LSTM Standard,1
2,14.455177,0.817420,0.923543,0.711296,0.012864+0.000000j,2.530732e-19+0.000000e+00j,LSTM Standard,2
3,15.880612,0.842864,0.939062,0.746667,0.010808+0.000000j,9.682378e-19+0.000000e+00j,LSTM Standard,3
4,17.029687,0.903391,0.965854,0.840928,0.014653+0.000000j,6.664056e-19+0.000000e+00j,LSTM Standard,4


# Plot Evaluation Metrics

## Boxplot Overview 

In [32]:
import seaborn as sns; sns.set(style='whitegrid', font_scale=1.2, font='helvetica')
%matplotlib qt

cols = df_aio.iloc[:, 0:6].columns
for y in cols:
    plt.figure()
    sns.boxplot(data=df_aio, x='method', y=y)
    plt.title(y.replace('_', ' ').title())
        

C:\Users\Lukas\Envs\esienv\lib\site-packages\numpy\core\_asarray.py:83: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
C:\Users\Lukas\Envs\esienv\lib\site-packages\numpy\core\_asarray.py:83: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
C:\Users\Lukas\Envs\esienv\lib\site-packages\numpy\core\_asarray.py:83: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
C:\Users\Lukas\Envs\esienv\lib\site-packages\numpy\core\_asarray.py:83: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
C:\Users\Lukas\Envs\esienv\lib\site-packages\numpy\core\_asarray.py:83: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
C:\Users\Lukas\

## Quadratic Scatter

In [82]:
import seaborn as sns; sns.set(style='whitegrid', font_scale=1.2, font='helvetica')
%matplotlib qt
methods_of_interest = ['Dense Standard', 'LSTM Standard']
# df_select = df_aio[df_aio['method'].str.contains('|'.join(methods_of_interest))]

cols = df_aio.iloc[:, 0:6].columns
for method_name in cols:

    vals_A = df_aio[df_aio['method'].str.contains(methods_of_interest[0])][method_name].values
    vals_B = df_aio[df_aio['method'].str.contains(methods_of_interest[1])][method_name].values
    d = {methods_of_interest[0]: vals_A, methods_of_interest[1]: vals_B,} 
    df_tmp = pd.DataFrame(d)
    plt.figure(figsize=(10, 10))
    ax = sns.scatterplot(data=df_tmp, x=methods_of_interest[0], y=methods_of_interest[1])


    xlim, ylim = (plt.xlim(), plt.ylim())
    plt.plot(xlim, ylim, '--k')
    xlim = (xlim[0]*0.95, xlim[1]*1.05)
    ylim = (ylim[0]*0.95, ylim[1]*1.05)
    plt.ylim(ylim)
    plt.xlim(xlim)
    
    # Title
    prop_higher = np.sum(vals_B > vals_A) / len(vals_A)
    cohens_d = (np.nanmean(vals_A) - np.nanmean(vals_B)) / np.mean([np.nanstd(vals_A), np.nanstd(vals_B)])
    median_diff = np.abs(np.nanmedian(vals_A-vals_B))
    method_name_title = method_name.replace('_', ' ').title()
    title = f'{method_name_title} ({methods_of_interest[1]} higher in {100*prop_higher:.1f} %)\nmedian_difference: {median_diff}\ncohens d: {abs(cohens_d):.2f}'
    plt.title(title)

    plt.gca().set_aspect('equal', adjustable='box')
    # break
del d, df_tmp

C:\Users\Lukas\Envs\esienv\lib\site-packages\numpy\core\_asarray.py:136: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order, subok=True)
C:\Users\Lukas\Envs\esienv\lib\site-packages\numpy\core\_asarray.py:136: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order, subok=True)


## Dependence on anything

In [193]:
from scipy.stats import pearsonr
variable = df.duration_of_trials.values
values = df.Dense_Standard_aucs_combined
plt.figure()
plt.scatter(variable, values, s=0.1)
plt.xlabel("Duration of Trials")
plt.ylabel("LSTM AUC Combined")
r, p = pearsonr(variable, values)
title = f"r={np.real(r):.2f}, p={p:.4f}"
plt.title(title)


Text(0.5, 1.0, 'r=-0.01, p=0.0961')

In [181]:
import pandas as pd
sns.set(font_scale=1.2, font='helvetica')
# pd.DataFrame( metrics , index=model_names)
target_column = 'duration_of_trials'
binning = True
n_bins = 8
stretched_indices = list(np.concatenate([[i]*src.data.shape[1] for i, src in enumerate(sim_test.source_data)]))

params = list(sim_test.simulation_info.columns)
series = [sim_test.simulation_info.iloc[stretched_indices][param] for param in params]
df = pd.concat(series, axis=1)
if binning:
    bins = np.linspace(df[target_column].min(), df[target_column].max()*1.01, num=n_bins)
    bin_labels = [str(round(bins[i], 1)) + ' - ' + str(round(bins[i+1], 1)) for i in range(len(bins)-1)]
    df['bins'] = np.digitize(df[target_column].values, bins=bins)
    target_column = 'bins'
else:
    bins = list(set(df[target_column].values))
    bins[-1] *= 1.01
    bin_labels = [str(bins[i]) for i in range(len(bins))]


for i, model_name in enumerate(list(set(df_aio.method.values))):
    if "lcmv" in model_name.lower():
        continue
    cols = df_aio[df_aio.method==model_name].iloc[:, 0:6].columns
    values = df_aio[df_aio.method==model_name].iloc[:, 0:6].values

    for metric_name, metric in zip(cols, values.T):
        col_name = model_name.replace(' ', '_') + '_' + metric_name.replace(' ', '_')
        df[col_name] = metric

print(df.columns)
dep_var_regex = target_column
dep_var_label = target_column.replace('_', ' ').title()
metric_names_nice = [col.replace('_', ' ').title() for col in df_aio.iloc[:, :6].columns]
for metric_name, metric_name_nice in zip(df_aio.iloc[:, :6].columns,  metric_names_nice):
    df_temp = pd.concat((df.filter(regex=dep_var_regex), df.filter(regex=metric_name)), axis=1).melt(dep_var_regex, var_name='cols', value_name='vals')
    g = sns.catplot(x=dep_var_regex, y='vals', hue='cols', capsize=.2, kind='point', data=df_temp)
    g.set(xticklabels=bin_labels, ylabel=metric_name_nice, xlabel=dep_var_label)
    g._legend.remove()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()

Index(['number_of_sources', 'positions', 'extents', 'amplitudes', 'shapes',
       'target_snr', 'betas', 'duration_of_trials', 'bins',
       'MNE_mean_localization_errors', 'MNE_aucs_combined', 'MNE_aucs_far',
       'MNE_aucs_close', 'MNE_nmses', 'MNE_mses',
       'LSTM_Standard_mean_localization_errors', 'LSTM_Standard_aucs_combined',
       'LSTM_Standard_aucs_far', 'LSTM_Standard_aucs_close',
       'LSTM_Standard_nmses', 'LSTM_Standard_mses',
       'ConvDip_Standard_mean_localization_errors',
       'ConvDip_Standard_aucs_combined', 'ConvDip_Standard_aucs_far',
       'ConvDip_Standard_aucs_close', 'ConvDip_Standard_nmses',
       'ConvDip_Standard_mses', 'Dense_Standard_mean_localization_errors',
       'Dense_Standard_aucs_combined', 'Dense_Standard_aucs_far',
       'Dense_Standard_aucs_close', 'Dense_Standard_nmses',
       'Dense_Standard_mses', 'eLORETA_mean_localization_errors',
       'eLORETA_aucs_combined', 'eLORETA_aucs_far', 'eLORETA_aucs_close',
       'eLORETA_nm

C:\Users\Lukas\Envs\esienv\lib\site-packages\numpy\core\_asarray.py:83: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
C:\Users\Lukas\Envs\esienv\lib\site-packages\numpy\core\_asarray.py:136: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order, subok=True)
C:\Users\Lukas\Envs\esienv\lib\site-packages\numpy\core\_asarray.py:83: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
C:\Users\Lukas\Envs\esienv\lib\site-packages\numpy\core\_asarray.py:136: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order, subok=True)
C:\Users\Lukas\Envs\esienv\lib\site-packages\numpy\core\_asarray.py:83: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, or

In [101]:
metrics

{'LSTM Standard':         mean_localization_errors  aucs_combined  aucs_far  aucs_close  \
 0                      23.515424       0.967474  0.999308    0.935640   
 1                      26.670565       0.796694  0.893222    0.700167   
 2                      14.455177       0.817420  0.923543    0.711296   
 3                      15.880612       0.842864  0.939062    0.746667   
 4                      17.029687       0.903391  0.965854    0.840928   
 ...                          ...            ...       ...         ...   
 101942                 13.268904       1.000000  1.000000    1.000000   
 101943                 11.976356       0.425000  0.437500    0.412500   
 101944                 19.464346       0.175000  0.237500    0.112500   
 101945                 11.537168       0.809917  0.842975    0.776860   
 101946                 11.682927       0.412500  0.462500    0.362500   
 
            nmses          mses         method  sample_id  
 0       0.008361  2.779794e-19  

In [83]:
import pandas as pd
sns.set(font_scale=1.2, font='helvetica')
# pd.DataFrame( metrics , index=model_names)
stretched_indices = list(np.concatenate([[i]*src.data.shape[1] for i, src in enumerate(sim_test.source_data)]))
target_column = 'target_snr'
binning = True
n_bins = 4

params = list(sim_test.simulation_info.columns)
series = [sim_test.simulation_info.iloc[stretched_indices][param] for param in params]
df = pd.concat(series, axis=1)
if binning:
    bins = np.linspace(df[target_column].min(), df[target_column].max()*1.01, num=n_bins)
    bin_labels = [str(int(round(bins[i]))) + ' - ' + str(int(round(bins[i+1]))) for i in range(len(bins)-1)]
    df['duration_bins'] = np.digitize(df[target_column].values, bins=bins)
    target_column = 'duration_bins'
else:
    bins = list(set(df[target_column].values))
    bins[-1] *= 1.01
    bin_labels = [str(bins[i]) for i in range(len(bins))]

for i, model_name in enumerate(model_names):
    for metric_name, metric in metrics.items():
        col_name = model_name.replace(' ', '_') + '_' + metric_name
        df[col_name] = metric[i]

print(df.columns)
dep_var_regex = target_column
dep_var_label = target_column.replace('_', ' ').title()
metric_names_nice = ['Mean Localization Error [mm]', 'AUC Far [%]', 'AUC Close [%]',  'AUC Combined [%]', 'Normalized Mean Squared Error', 'Mean Squared Error']
for metric_name, metric_name_nice in zip(metrics.keys(), metric_names_nice):
    df_temp = pd.concat((df.filter(regex=dep_var_regex), df.filter(regex=metric_name)), axis=1).melt(dep_var_regex, var_name='cols', value_name='vals')
    g = sns.catplot(x=dep_var_regex, y='vals', hue='cols', capsize=.2, kind='point', data=df_temp)
    g.set(xticklabels=bin_labels, ylabel=metric_name_nice, xlabel=dep_var_label)
    g._legend.remove()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()

KeyError: 0

In [ ]:
import pandas as pd
sns.set(font_scale=1.2, font='helvetica')
# pd.DataFrame( metrics , index=model_names)
stretched_indices = np.repeat(np.arange(sim_test.n_samples), np.mean([src.data.shape[1] for src in sim_test.source_data]))
snr_bins = (2, 4, 6, 8, 10, 20)
# snr_bins = (0., 10, 20)

snr_bin_labels = [str(snr_bins[i]) + ' - ' + str(snr_bins[i+1]) for i in range(len(snr_bins)-1)]
params = list(sim_test.simulation_info.columns)
series = [sim_test.simulation_info.iloc[stretched_indices][param] for param in params]
df = pd.concat(series, axis=1)
df['target_snr_bins'] = np.digitize(df.target_snr, bins=snr_bins)
# df['avg_eccentricity'] = [np.median((df.positions.values[i]**2).sum(axis=1)**(1/2)) for i in range(df.shape[0])]
# df['avg_extent'] = [np.mean(df.extents.values[i]) for i in range(df.shape[0])]
# df['avg_beta'] = [np.mean(df.betas.values[i]) for i in range(df.shape[0])]

for i, model_name in enumerate(model_names):
    for metric_name, metric in metrics.items():
        col_name = model_name + '_' + metric_name

        df[col_name] = metric[i]

print(df.columns)
metric_names_nice = ['Mean Localization Error [mm]', 'AUC Far [%]', 'AUC Close [%]', 'Normalized Mean Squared Error', 'Mean Squared Error']
for metric_name, metric_name_nice in zip(metrics.keys(), metric_names_nice):
    df_temp = pd.concat((df.filter(regex='snr_'), df.filter(regex=metric_name)), axis=1).melt('target_snr_bins', var_name='cols', value_name='vals')
    g = sns.catplot(x='target_snr_bins', y='vals', hue='cols', capsize=.2, kind='point', data=df_temp)
    g.set(xticklabels=snr_bin_labels, ylabel=metric_name_nice, xlabel='Signal to Noise Ratio (SNR)')
    g._legend.remove()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()

## Dependence on eccentricity

In [ ]:
import pandas as pd
sns.set(font_scale=1.2, font='helvetica')
# pd.DataFrame( metrics , index=model_names)
stretched_indices = np.repeat(np.arange(sim_test.n_samples), np.mean([src.data.shape[1] for src in sim_test.source_data]))



params = list(sim_lstm_test.simulation_info.columns)
series = [sim_lstm_test.simulation_info.iloc[stretched_indices[choice]][param] for param in params]
df = pd.concat(series, axis=1)
df['avg_eccentricity'] = [np.median((df.positions.values[i]**2).sum(axis=1)**(1/2)) for i in range(df.shape[0])]
ecc_bins = np.linspace(int(df['avg_eccentricity'].values.min()), np.ceil(df['avg_eccentricity'].values.max()).astype(int), num=5)
ecc_bin_labels = [str(ecc_bins[i]) + ' - ' + str(ecc_bins[i+1]) for i in range(len(ecc_bins)-1)]
df['avg_eccentricity_bins'] = np.digitize(df['avg_eccentricity'].values, bins=ecc_bins)

df['avg_extent'] = [np.mean(df.extents.values[i]) for i in range(df.shape[0])]
df['avg_beta'] = [np.mean(df.betas.values[i]) for i in range(df.shape[0])]

for i, model_name in enumerate(model_names):
    for metric_name, metric in metrics.items():
        col_name = model_name + '_' + metric_name

        df[col_name] = metric[i]

print(df.columns)
dep_var_regex = 'avg_eccentricity_bins'  # 'target_snr_bins'
dep_var_label = 'Average eccentricity'  # 'Signal to Noise Ratio (SNR)'
metric_names_nice = ['Mean Localization Error [mm]', 'AUC Far [%]', 'AUC Close [%]', 'Normalized Mean Squared Error', 'Mean Squared Error']
for metric_name, metric_name_nice in zip(metrics.keys(), metric_names_nice):
    df_temp = pd.concat((df.filter(regex=dep_var_regex), df.filter(regex=metric_name)), axis=1).melt(dep_var_regex, var_name='cols', value_name='vals')
    g = sns.catplot(x=dep_var_regex, y='vals', hue='cols', capsize=.2, kind='point', data=df_temp)
    g.set(xticklabels=ecc_bin_labels, ylabel=metric_name_nice, xlabel=dep_var_label)
    g._legend.remove()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()

## Dependence on No. of sources

In [ ]:
import pandas as pd
sns.set(font_scale=1.2, font='helvetica')
# pd.DataFrame( metrics , index=model_names)
stretched_indices = list(np.concatenate([[i]*src.data.shape[1] for i, src in enumerate(sim_test.source_data)]))

# nsrc_bins = (1, 2, 4, 8, 16, 20)
nsrc_bins = (1, 2, 4, 8, 10)

nsrc_bin_labels = [str(nsrc_bins[i]) + ' - ' + str(nsrc_bins[i+1]) for i in range(len(nsrc_bins)-1)]

params = list(sim_test.simulation_info.columns)
series = [sim_test.simulation_info.iloc[stretched_indices][param] for param in params]
df = pd.concat(series, axis=1)
df['number_of_sources_bins'] = np.digitize(df['number_of_sources'].values, bins=nsrc_bins)

for i, model_name in enumerate(model_names):
    for metric_name, metric in metrics.items():
        col_name = model_name + '_' + metric_name
        df[col_name] = metric[i]

print(df.columns)
dep_var_regex = 'number_of_sources_bins'  # 'target_snr_bins'
dep_var_label = 'Number of True Sources'  # 'Signal to Noise Ratio (SNR)'
metric_names_nice = ['Mean Localization Error [mm]', 'AUC Far [%]', 'AUC Close [%]', 'Normalized Mean Squared Error', 'Mean Squared Error']
for metric_name, metric_name_nice in zip(metrics.keys(), metric_names_nice):
    df_temp = pd.concat((df.filter(regex=dep_var_regex), df.filter(regex=metric_name)), axis=1).melt(dep_var_regex, var_name='cols', value_name='vals')
    g = sns.catplot(x=dep_var_regex, y='vals', hue='cols', capsize=.2, kind='point', data=df_temp)
    g.set(xticklabels=nsrc_bin_labels, ylabel=metric_name_nice, xlabel=dep_var_label)
    g._legend.remove()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()


## Dependence on Beta

In [ ]:
import pandas as pd
sns.set(font_scale=1.2, font='helvetica')
# pd.DataFrame( metrics , index=model_names)
stretched_indices = np.repeat(np.arange(sim_lstm_test.n_samples), sim_lstm_test.source_data[0].data.shape[1])
snr_bins = (0.5, 2, 4, 6, 8, 10)
snr_bin_labels = [str(snr_bins[i]) + ' - ' + str(snr_bins[i+1]) for i in range(len(snr_bins)-1)]

beta_bins = (0.5, 0.75, 1, 1.25, 1.5)
beta_bin_labels = [str(beta_bins[i]) + ' - ' + str(beta_bins[i+1]) for i in range(len(beta_bins)-1)]


params = list(sim_lstm_test.simulation_info.columns)
series = [sim_lstm_test.simulation_info.iloc[stretched_indices[choice]][param] for param in params]
df = pd.concat(series, axis=1)
df['target_snr_bins'] = np.digitize(df.target_snr, bins=snr_bins)

df['avg_beta'] = [np.mean(df.betas.values[i]) for i in range(df.shape[0])]
df['avg_beta_bins']  = np.digitize(df.avg_beta, bins=beta_bins)


for i, model_name in enumerate(model_names):
    for metric_name, metric in metrics.items():
        col_name = model_name + '_' + metric_name

        df[col_name] = metric[i]

print(df.columns)
regex = 'beta_'
metric_names_nice = ['Mean Localization Error [mm]', 'AUC Far [%]', 'AUC Close [%]', 'Normalized Mean Squared Error', 'Mean Squared Error']
for metric_name, metric_name_nice in zip(metrics.keys(), metric_names_nice):
    df_temp = pd.concat((df.filter(regex=regex), df.filter(regex=metric_name)), axis=1).melt('avg_beta_bins', var_name='cols', value_name='vals')
    g = sns.catplot(x='avg_beta_bins', y='vals', hue='cols', capsize=.2, kind='point', data=df_temp)
    g.set(xticklabels=beta_bin_labels, ylabel=metric_name_nice, xlabel='Beta Exponent')
    g._legend.remove()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()

## Dependence on Extent

In [ ]:
import pandas as pd
sns.set(font_scale=1.2, font='helvetica')
# pd.DataFrame( metrics , index=model_names)
stretched_indices = np.repeat(np.arange(sim_lstm_test.n_samples), sim_lstm_test.source_data[0].data.shape[1])

extent_bins = np.linspace(20, 40, 5)
extent_bin_labels = [str(int(round(extent_bins[i]))) + ' - ' + str(int(round(extent_bins[i+1]))) for i in range(len(extent_bins)-1)]

params = list(sim_lstm_test.simulation_info.columns)
series = [sim_lstm_test.simulation_info.iloc[stretched_indices[choice]][param] for param in params]
df = pd.concat(series, axis=1)

df['avg_extent'] = [np.mean(df.extents.values[i]) for i in range(df.shape[0])]
df['avg_extent_bins']  = np.digitize(df.avg_extent, bins=extent_bins)


for i, model_name in enumerate(model_names):
    for metric_name, metric in metrics.items():
        col_name = model_name + '_' + metric_name

        df[col_name] = metric[i]

print(df.columns)
regex = 'extent_'
metric_names_nice = ['Mean Localization Error [mm]', 'AUC Far [%]', 'AUC Close [%]', 'Normalized Mean Squared Error', 'Mean Squared Error']
for metric_name, metric_name_nice in zip(metrics.keys(), metric_names_nice):
    df_temp = pd.concat((df.filter(regex=regex), df.filter(regex=metric_name)), axis=1).melt('avg_extent_bins', var_name='cols', value_name='vals')
    g = sns.catplot(x='avg_extent_bins', y='vals', hue='cols', capsize=.2, kind='point', data=df_temp)
    g.set(xticklabels=extent_bin_labels, ylabel=metric_name_nice, xlabel='Source extent [mm]')
    g._legend.remove()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()

## Save all figures

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

def multipage(filename, figs=None, dpi=300, png=False):
    ''' Saves all open (or list of) figures to filename.pdf with dpi''' 
    pp = PdfPages(filename)
    path = os.path.dirname(filename)
    fn = os.path.basename(filename)[:-4]

    if figs is None:
        figs = [plt.figure(n) for n in plt.get_fignums()]
    for i, fig in enumerate(figs):
        print(f'saving fig {fig}\n')
        fig.savefig(pp, format='pdf', dpi=dpi)
        if png:
            fig.savefig(f'{path}\\{i}_{fn}.png', dpi=600)
    pp.close()

multipage(f'figures\\figs_5000_10samples\\figs_5000_10samples.pdf', png=True)

In [ ]:
import seaborn as sns
%matplotlib qt
model_names[0] = 'Dense'

data = {model_name: nmse for model_name, nmse in zip(model_names, nmses)}
plt.figure()
sns.kdeplot(data=data, multiple='stack')
plt.title('Normalized Mean Squared Errors')

plt.figure()
data = {model_name: mse for model_name, mse in zip(model_names, mses)}
sns.kdeplot(data=data, multiple='stack')
plt.title('Mean Squared Errors')


plt.figure()
data = {model_name: auc for model_name, auc in zip(model_names, aucs_far)}
sns.kdeplot(data=data, multiple='stack')
plt.title('Far area under the curve')


plt.figure()
data = {model_name: auc for model_name, auc in zip(model_names, aucs_close)}
sns.kdeplot(data=data, multiple='stack')
plt.title('Close area under the curve')

plt.figure()
data = {model_name: mle for model_name, mle in zip(model_names, mean_localization_errors)}
sns.kdeplot(data=data, multiple='stack')
plt.title('Close area under the curve')
plt.title('Mean Localization Errors')

plt.tight_layout()

# Performance per Difficulty Evaluation

In [ ]:
metrics = [nmses, mean_localization_errors, aucs_far, aucs_close]
metric_names = ['nmses', 'mean_localization_errors', 'aucs_far', 'aucs_close']

covariates = ['target_snr', 'number_of_sources']

n_samples = int(np.array(nmses).shape[1] / 20)
for covariate in covariates:
    sim_params = sim_lstm_test.simulation_info[covariate].values[:n_samples]
    sim_params = np.repeat(sim_params, 20)
    for metric, metric_name in zip(metrics, metric_names):
        plt.figure()
        for i, model_name in enumerate(model_names):
            plt.scatter(sim_params, metric[i], label=model_name, s=.7)
        plt.title(f'{covariate}, {metric_name}')
        plt.legend()


## Speed test

In [ ]:
import time

def new_sim_params(sr=100, packages_per_second=20):
    package_size = int( round( sr / packages_per_second  ) )
    package_interval = package_size/sr

    n_chan = len(sim_lstm_test.eeg_data.ch_names)
    data_package = np.random.randn(n_chan, package_size)

    sim_data_package = Simulation(fwd, info, settings=dict(duration_of_trial=0.01*package_size)).simulate(1)
    print(f'performing predictions {packages_per_second} times per second')

    return sim_data_package, package_interval

packages_per_second = 50
sim_data_package, package_interval = new_sim_params(packages_per_second=packages_per_second)

while True:
    start = time.time()
    # stc = net_dense.predict(sim_data_package)
    stc = net_lstm.predict(sim_data_package)

    stop = time.time()
    diff = stop-start
    if stop-start > package_interval:
        print(f"took longer than expected: {diff} (instead of {package_interval})")
        print(f'decreasing package interval by one')
        packages_per_second -= 1
        sim_data_package, package_interval = new_sim_params(packages_per_second=packages_per_second)
        print(f'packages_per_second={packages_per_second}\n')
        continue
    time.sleep(package_interval-diff)

